In [10]:
# Suppose I take O, opponent take X, 0 means nothing, 1 means O, 2 means X.
grid = [[0,1,2]] * 9
# Value: 0 if intermediate or , 1 if win, -1 if lose
# Policy: assign 1 to one of the empty square
# Assumption: opponent using same policy function, ie. always end up tie.

0